# Qiita記事のタイトルを自動生成するモデルの学習

事前学習済み日本語T5モデルを、今回の自動生成タスク用に転移学習（ファインチューニング）します。

- **T5（Text-to-Text Transfer Transformer）**: テキストを入力されるとテキストを出力するという統一的枠組みで様々な自然言語処理タスクを解く深層学習モデル（[解説](https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part7.html)）
- **事前学習**: 個別のタスク用に学習をする前に文法や一般的な言葉の意味を学習させること（教師なし学習や自己教師あり学習とWikipedia等の大規模データ（コーパス）を用いることで広く一般的な知識を持ったモデルを作れる）
- **転移学習、ファインチューニング**: 事前学習済みモデルを初期値にして、特定のタスク用に追加で学習を行うこと（主に教師あり学習）


今回は入出力が次の形式を持ったタスク用に転移学習します。
- **入力**: "body: {body}"をトークナイズしたトークンID列
- **出力**: "title: {title}"をトークナイズしたトークンID列

ここで、{body}は記事本文（を前処理したもの）、{title}はタイトルです。

### 補足

今回はタイトル自動生成タスク用のモデルを作りますが、データセット（TSVファイル）を他のタスク用のものに入れ替えれば、ほぼコードを変えることなく（最大入出力トークン数を指定するmax_input_lengthとmax_target_lengthは適切な値に変える必要があるでしょう）、他のタスク用のモデルを作ることもできます。ぜひ色々試してみてください。

## ライブラリの準備

### パスワード設定

**※学習に用いるデータセットとモデルの解凍にはパスワードが必要です（一般には公開しません）。**  
**※次の変数 RESOURCES_PASSWORD に解凍用の秘密のパスワードを設定してから、以降の処理を実行してください。**

In [1]:
# データセットとモデルの解凍用パスワード
RESOURCES_PASSWORD = ""

### 依存ライブラリのインストール

In [2]:
!pip install -q torch==1.12.* torchtext==0.13.* torchvision==0.13.* torchaudio==0.12.*
!pip install -q transformers==4.20.1 pytorch_lightning==1.7.1 sentencepiece

     |████████████████████████████████| 4.4 MB 13.7 MB/s 
     |████████████████████████████████| 701 kB 56.2 MB/s 
     |████████████████████████████████| 1.3 MB 51.9 MB/s 
     |████████████████████████████████| 6.6 MB 39.0 MB/s 
     |████████████████████████████████| 596 kB 49.5 MB/s 
     |████████████████████████████████| 101 kB 11.9 MB/s 
     |████████████████████████████████| 141 kB 74.7 MB/s 
     |████████████████████████████████| 419 kB 73.3 MB/s 
     |████████████████████████████████| 5.9 MB 47.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.


### 学習に利用するデータセットと事前学習済みモデルをダウンロード

In [3]:
!gdown --id "1o3Yzm2oZxqGMXuQsR2vhh3KUVFcJRadv"

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1o3Yzm2oZxqGMXuQsR2vhh3KUVFcJRadv
To: /content/qiita_hit_items.json.zip
100% 72.5M/72.5M [00:00<00:00, 73.1MB/s]


### データと事前学習済みモデルの解凍

In [4]:
!unzip -P {RESOURCES_PASSWORD} qiita_hit_items.json.zip

Archive:  qiita_hit_items.json.zip
  inflating: qiita_hit_items.json    


### 各種ディレクトリの作成

- data: 学習データの置き場所
- model: 学習済みモデルの出力先

In [5]:
!mkdir -p /content/model
!mkdir -p /content/data

In [6]:
OUTPUT_MODEL_DIR = "/content/model"

※/content/modelディレクトリは、このColabインスタンスがリセットされたら削除されます。  
削除されない場所に保存したいのであれば、Google Driveに保存するのが簡単です。

ColabからGoogle Driveを利用する方法: https://qiita.com/kado_u/items/45b76f9a6f920bf0f786

In [7]:
# もし、Google Driveに学習済みモデルを保存する場合は次の処理を実行
# ※Googleアカウントの認証が必要です。

# from google.colab import drive
# drive.mount('/content/drive')

# OUTPUT_MODEL_DIR = "/content/drive/MyDrive/qiita_title_generation_model"

# !mkdir -p {OUTPUT_MODEL_DIR}

## 学習可能な形式にデータを変換

### 前処理の定義

#### 文字列の正規化

表記揺れを減らします。今回は[neologdの正規化処理](https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja)を一部改変したものを利用します。
処理の詳細はリンク先を参照してください。

In [8]:
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

#### Markdownのクリーニング

タイトルを考える上で関係のなさそうな文章を削る処理を行います。  
以下のノイズとなるデータを削除し、タブや改行を空白文字にしたり、文字を小文字に揃える等の処理を行います。

- ソースコード
- URLやリンク
- 画像

現状、img以外のHTML要素を残していますが、タイトルに関係なさそうな要素を削ると精度が上がるかもしれません。

In [9]:
import re

CODE_PATTERN = re.compile(r"```.*?```", re.MULTILINE | re.DOTALL)
LINK_PATTERN = re.compile(r"!?\[([^\]\)]+)\]\([^\)]+\)")
IMG_PATTERN = re.compile(r"<img[^>]*>")
URL_PATTERN = re.compile(r"(http|ftp)s?://[^\s]+")
NEWLINES_PATTERN = re.compile(r"(\s*\n\s*)+")

def clean_markdown(markdown_text):
    markdown_text = CODE_PATTERN.sub(r"", markdown_text)
    markdown_text = LINK_PATTERN.sub(r"\1", markdown_text)
    markdown_text = IMG_PATTERN.sub(r"", markdown_text)
    markdown_text = URL_PATTERN.sub(r"", markdown_text)
    markdown_text = NEWLINES_PATTERN.sub(r"\n", markdown_text)
    markdown_text = markdown_text.replace("`", "")
    return markdown_text

def normalize_text(markdown_text):
    markdown_text = clean_markdown(markdown_text)
    markdown_text = markdown_text.replace("\t", " ")
    markdown_text = normalize_neologd(markdown_text).lower()
    markdown_text = markdown_text.replace("\n", " ")
    return markdown_text

def preprocess_qiita_body(markdown_text):
    return "body: " + normalize_text(markdown_text)[:4000]

def preprocess_qiita_title(markdown_text):
    # return normalize_text(markdown_text)
    return "title: " + normalize_text(markdown_text)

### Qiita記事データ（JSON形式）を学習データ（TSV形式）に変換

In [10]:
import json
import gzip
import random
import math

# Qiita記事データ（JSON形式）を読み込む
with open("/content/qiita_hit_items.json", 
               "rt", encoding="utf8") as f_in:
    dataset = json.load(f_in)

# 前処理を施す
for data in dataset:
    if "body" not in data:
        continue

    body = data["body"]
    title = data["title"]
    data["preprocessed_body"] = preprocess_qiita_body(body)
    data["preprocessed_title"] = preprocess_qiita_title(title)

# 再現性のため、ランダムシードを固定してデータをシャッフルする
random.seed(42)
random.shuffle(dataset)

# 学習データを訓練データ/開発（バリデーション）データ/テスト（評価）データに分割する。
total_count = len(dataset)
train_count = math.ceil(total_count * 0.92)
dev_count = math.ceil(total_count * 0.04)
test_count = total_count - train_count - dev_count

with open("/content/data/train.tsv", "w", encoding="utf8") as f_train, \
    open("/content/data/dev.tsv", "w", encoding="utf8") as f_dev, \
    open("/content/data/test.tsv", "w", encoding="utf8") as f_test:

    for i, data in enumerate(dataset):
        if "body" not in data:
            continue
        
        preprocessed_body = data["preprocessed_body"]
        preprocessed_title = data["preprocessed_title"]

        if i < train_count:
            f_out = f_train
        elif i < train_count + dev_count:
            f_out = f_dev
        else:
            f_out = f_test

        f_out.write(f"{preprocessed_body}\t{preprocessed_title}\n")

print(train_count, dev_count, test_count)

8832 384 384


## 学習に必要なクラス等の定義

学習にはPyTorch/PyTorch-lightning/Transformersを利用します。

In [11]:
import argparse
import os
import random
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import AdamW,get_linear_schedule_with_warmup

# 乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(12345)

In [12]:
# 事前学習済みモデルの置き場所
PRETRAINED_MODEL_PATH = "sonoisa/t5-base-japanese"

# GPU利用有無
USE_GPU = torch.cuda.is_available()

# 各種ハイパーパラメータ
args_dict = dict(
    data_dir="/content/data",  # データセットのディレクトリ
    output_dir=OUTPUT_MODEL_DIR,  # 学習済みモデルの出力先
    model_name_or_path=PRETRAINED_MODEL_PATH,
    tokenizer_name_or_path=PRETRAINED_MODEL_PATH,

    learning_rate=1e-3,  # 学習率
    adam_epsilon=1e-8,
    weight_decay=0.0,
    warmup_steps=0,
    gradient_accumulation_steps=1,

    # max_input_length=512,
    # max_target_length=64,
    # train_batch_size=8,
    # eval_batch_size=8,
    # num_train_epochs=4,

    n_gpu=1 if USE_GPU else 0,
    early_stop_callback=False,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0,
    seed=12345,
)


### TSVデータセットクラス

TSV形式のファイルをデータセットとして読み込みます。  
形式は"{入力される文字列}\t{出力される文字列}"です。

In [13]:
class TsvDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, input_max_len=512, target_max_len=512):
        self.file_path = os.path.join(data_dir, type_path)
    
        self.input_max_len = input_max_len
        self.target_max_len = target_max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()
  
    def __len__(self):
        return len(self.inputs)
  
    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        source_mask = self.inputs[index]["attention_mask"].squeeze()
        target_mask = self.targets[index]["attention_mask"].squeeze()

        return {"source_ids": source_ids, "source_mask": source_mask, 
                "target_ids": target_ids, "target_mask": target_mask}
  
    def _build(self):
        # ファイルからデータを読み込む
        with open(self.file_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip().split("\t")
                assert len(line) == 2
                assert len(line[0]) > 0
                assert len(line[1]) > 0

                inputs = [line[0]]
                targets = [line[1]]

                tokenized_inputs = self.tokenizer.batch_encode_plus(
                    inputs, max_length=self.input_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = self.tokenizer.batch_encode_plus(
                    targets, max_length=self.target_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                self.inputs.append(tokenized_inputs)
                self.targets.append(tokenized_targets)

試しにテストデータ（test.tsv）を読み込み、トークナイズ結果をみてみます。

In [14]:
# トークナイザー（SentencePiece）モデルの読み込み
tokenizer = T5Tokenizer.from_pretrained(PRETRAINED_MODEL_PATH, is_fast=True)

# テストデータセットの読み込み
train_dataset = TsvDataset(tokenizer, args_dict["data_dir"], 
                           "test.tsv", 
                           input_max_len=512, target_max_len=64)

Downloading:   0%|          | 0.00/785k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/710 [00:00<?, ?B/s]

テストデータの1レコード目をみてみます。

In [15]:
for data in train_dataset:
    print("A. 入力データの元になる文字列")
    print(tokenizer.decode(data["source_ids"]))
    print()
    print("B. 入力データ（Aの文字列がトークナイズされたトークンID列）")
    print(data["source_ids"])
    print()
    print("C. 出力データの元になる文字列")
    print(tokenizer.decode(data["target_ids"]))
    print()
    print("D. 出力データ（Cの文字列がトークナイズされたトークンID列）")
    print(data["target_ids"])
    break

A. 入力データの元になる文字列
body: primaryキーまたはuniqueキーを指定して、レコードがなければinsert、あればupdateしたい。 #mysql ##insert... on duplicate key update文を使う mysql>desc foo; +-------+--------------+------+-----+---------+-------+ |field|type|null|key|default|extra| +-------+--------------+------+-----+---------+-------+ |code|varchar(32)|no|pri||| |name|varchar(256)|yes||null|| +-------+--------------+------+-----+---------+-------+ mysql>insert into foo(code,name)values('0001', 'abc')on duplicate key update code='0001',name='def'; code='0001'のレコードがないので、insert mysql>select*from foo; +------+------+ |code|name| +------+------+ |0001|abc| +------+------+ mysql>insert into foo(code,name)values('0001', 'abc')on duplicate key update code='0001',name='def'; code='0001'のレコードがあるので、name='def'でupdate +------+------+ |code|name| +------+------+ |0001|def| +------+------+ #oracle ##merge文を使う(9i以降) ex. merge into user_master a using(select '0056ddgd456' as user_id,700 as user_point from dual)b on(a.user_id=b.user_id) when matched

### 学習処理クラス

[PyTorch-Lightning](https://github.com/PyTorchLightning/pytorch-lightning)を使って学習します。

PyTorch-Lightningとは、機械学習の典型的な処理を簡潔に書くことができるフレームワークです。

In [16]:
import os

class T5FineTuner(pl.LightningModule):

    def __init__(self, hparams):
        super().__init__()
        self.save_hyperparameters(hparams)
        # 事前学習済みモデルの読み込み
        self.model = T5ForConditionalGeneration.from_pretrained(self.hparams.model_name_or_path)
        # トークナイザーの読み込み
        self.tokenizer = T5Tokenizer.from_pretrained(self.hparams.tokenizer_name_or_path, is_fast=True)

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, 
                decoder_attention_mask=None, labels=None):
        """順伝搬"""
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

    def _step(self, batch):
        """ロス計算"""
        labels = batch["target_ids"]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch["target_mask"],
            labels=labels
        )

        loss = outputs[0]
        return loss

    def training_step(self, batch, batch_idx):
        """訓練ステップ処理"""
        loss = self._step(batch)
        self.log("train_loss", loss)
        return {"loss": loss}

    def _precision_recall_f1_step(self, batch):
        """トークンレベルの一致率（precision/recall/f1）を求める"""
        # ※文章の生成の場合、BLEUやROUGEの方がより良い場合もある。

        input_ids = batch["source_ids"]
        input_mask = batch["source_mask"]
        targets = batch["target_ids"]

        outputs = self.model.generate(
            input_ids=input_ids, 
            attention_mask=input_mask, 
            max_length=self.hparams.max_target_length,
            repetition_penalty=1.5,
            )
        outputs = [set(output.cpu().tolist()) for output in outputs]
        targets = [set(target.cpu().tolist()) for target in targets]

        precisions = [len(o & t) / len(o) for o, t in zip(outputs, targets)]
        recalls = [len(o & t) / len(t) for o, t in zip(outputs, targets)]
        f1s = [2 * p * r / (p + r) if p + r > 0 else 0 for p, r in zip(precisions, recalls)]

        p = precisions
        r = recalls
        f1 = f1s

        return p, r, f1

    def validation_step(self, batch, batch_idx):
        """バリデーションステップ処理"""
        loss = self._step(batch)
        p, r, f1 = self._precision_recall_f1_step(batch)
        self.log("val_loss", loss)
        return {"val_loss": loss, "val_p": p, "val_r": r, "val_f1": f1}

    def validation_epoch_end(self, outputs):
        """バリデーション完了処理"""
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        avg_p = np.mean([v for x in outputs for v in x["val_p"]])
        avg_r = np.mean([v for x in outputs for v in x["val_r"]])
        avg_f1 = np.mean([v for x in outputs for v in x["val_f1"]])

        self.log("val_loss", avg_loss, prog_bar=True)
        self.log("val_p", avg_p, prog_bar=True)
        self.log("val_r", avg_r, prog_bar=True)
        self.log("val_f1", avg_f1, prog_bar=True)

        return {"val_loss": avg_loss, "val_p": avg_p, "val_r": avg_r, "val_f1": avg_f1}

    def test_step(self, batch, batch_idx):
        """テストステップ処理"""
        loss = self._step(batch)
        p, r, f1 = self._precision_recall_f1_step(batch)
        self.log("test_loss")
        return {"test_loss": loss, "test_p": p, "test_r": r, "test_f1": f1}

    def test_epoch_end(self, outputs):
        """テスト完了処理"""
        avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        avg_p = torch.stack([x["test_p"] for x in outputs]).mean()
        avg_r = torch.stack([x["test_r"] for x in outputs]).mean()
        avg_f1 = torch.stack([x["test_f1"] for x in outputs]).mean()

        self.log("test_loss", avg_loss, prog_bar=True)
        self.log("test_p", avg_p, prog_bar=True)
        self.log("test_r", avg_r, prog_bar=True)
        self.log("test_f1", avg_f1, prog_bar=True)

    def configure_optimizers(self):
        """オプティマイザーとスケジューラーを作成する"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() 
                             if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() 
                             if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, 
                          lr=self.hparams.learning_rate, 
                          eps=self.hparams.adam_epsilon)

        # scheduler = get_linear_schedule_with_warmup(
        #     optimizer, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=self.t_total
        # )

        # return [optimizer], [{"scheduler": scheduler, "interval": "step", "frequency": 1}]
        return [optimizer]

    def get_dataset(self, tokenizer, type_path, args):
        """データセットを作成する"""
        return TsvDataset(
            tokenizer=tokenizer, 
            data_dir=args.data_dir, 
            type_path=type_path, 
            input_max_len=args.max_input_length,
            target_max_len=args.max_target_length)
    
    def setup(self, stage=None):
        """初期設定（データセットの読み込み）"""
        if stage == "fit" or stage is None:
            train_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                            type_path="train.tsv", args=self.hparams)
            self.train_dataset = train_dataset

            val_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                            type_path="dev.tsv", args=self.hparams)
            self.val_dataset = val_dataset

            # self.t_total = (
            #     (len(train_dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
            #     // self.hparams.gradient_accumulation_steps
            #     * float(self.hparams.num_train_epochs)
            # )

    def train_dataloader(self):
        """訓練データローダーを作成する"""
        return DataLoader(self.train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=4)

    def val_dataloader(self):
        """バリデーションデータローダーを作成する"""
        return DataLoader(self.val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)


Transformersのモデルのみを保存するようにする


In [17]:
import os

class TransformerTrainer(pl.Trainer):
    def save_checkpoint(self, filepath, weights_only=False):
        """モデルの重みを保存する"""
        # 学習済みモデル（とトークナイザー）を保存する
        if self.is_global_zero:
            print(f"save model: {filepath}")
            model_dir_name = os.path.dirname(filepath)
            lightningmodel = self.model
            lightningmodel.model.save_pretrained(model_dir_name)
            lightningmodel.tokenizer.save_pretrained(model_dir_name)

## 転移学習を実行

In [18]:
# 学習に用いるハイパーパラメータを設定する
args_dict.update({
    "max_input_length":  512,  # 入力文の最大トークン数 (<512)
    "max_target_length": 64,  # 出力文の最大トークン数 (<512)
    "train_batch_size":  8,
    "eval_batch_size":   8,
    "num_train_epochs":  4,  # 適切なエポック数にしてください。2～10くらいのはず。
    })
args = args_dict

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    args["output_dir"], 
    # バリデーションセット（dev.tsv）におけるF1値が最大のモデルを1個だけ保存します。
    monitor="val_f1", mode="max", save_top_k=1
)

train_params = dict(
    gpus=args["n_gpu"],
    max_epochs=args["num_train_epochs"],
    precision= 16 if args["fp_16"] else 32,
    # amp_level=args["opt_level"],
    gradient_clip_val=args["max_grad_norm"],
    accumulate_grad_batches=args["gradient_accumulation_steps"],
    callbacks=[checkpoint_callback],
)

In [19]:
# 転移学習の実行（GPUを利用すれば1エポック10分程度）
model = T5FineTuner(args)
trainer = TransformerTrainer(**train_params)
trainer.fit(model)

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:446: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training: 0it [00:00, ?it/s]

...

save model: /content/model/epoch=1-step=2132.ckpt


Validation: 0it [00:00, ?it/s]

save model: /content/model/epoch=2-step=3198.ckpt


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=4` reached.


## 学習済みモデルの読み込み

In [20]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer

# トークナイザー（SentencePiece）
tokenizer = T5Tokenizer.from_pretrained(OUTPUT_MODEL_DIR, is_fast=True)

# 学習済みモデル
trained_model = T5ForConditionalGeneration.from_pretrained(OUTPUT_MODEL_DIR)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    trained_model.cuda()

## 全テストデータに対してタイトル生成を実行

In [21]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

# テストデータの読み込み
test_dataset = TsvDataset(tokenizer, args_dict["data_dir"], 
                          "test.tsv", 
                          input_max_len=args["max_input_length"], 
                          target_max_len=args["max_target_length"])

test_loader = DataLoader(test_dataset, batch_size=4, num_workers=4)

# 推論モード
trained_model.eval()

outputs = []
targets = []
for batch in tqdm(test_loader):
    input_ids = batch['source_ids']
    input_mask = batch['source_mask']
    if USE_GPU:
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()

    outs = trained_model.generate(input_ids=input_ids, 
        attention_mask=input_mask, 
        max_length=args["max_target_length"],
        repetition_penalty=1.5,
        )

    dec = [tokenizer.decode(ids, skip_special_tokens=True, 
                          clean_up_tokenization_spaces=False) 
                        for ids in outs]
    target = [tokenizer.decode(ids, skip_special_tokens=True, 
                             clean_up_tokenization_spaces=False) 
                        for ids in batch["target_ids"]]


    output_idset = [set(ids.cpu()) for ids in outs]
    output_idset = [set(ids) for ids in outs]

    outputs.extend(dec)
    targets.extend(target)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/92 [00:00<?, ?it/s]

In [22]:
def postprocess_title(title):
  return re.sub(r"^title: ", "", title)

generatedが自動生成されたタイトル、actualが実際に人が付けたタイトルです。

In [23]:
import re

for output, target in list(zip(outputs, targets)):
  print(f"generated: {postprocess_title(output)}")
  print(f"actual:    {postprocess_title(target)}")
  print()

generated: mysqlでinsert,update
actual:    レコードがなければinsert,あればupdateをするsql

generated: coreos入門
actual:    coreos入門

generated: 乃木坂46の顔をcnnで分類してみた
actual:    pytorchを使って日向坂46の顔分類をしよう!

generated: 瞬殺で作るapache mesos+chronos+marathon+docker
actual:    瞬殺で作るmesos+chronos+marathon+dockerクラスタ環境

generated: django入門第1章(django編)
actual:    djangoを最速でマスターするpart2

generated: 機械学習コース修了者へ
actual:    機械学習のコースを修了したのでオススメしてみます

generated: 手作りmean stack(mongodb+express+node.js)を試してみた
actual:    mongodb+express+angularjs+node.jsでシンプルなcrudアプリ作成

generated: mac os xにrbenvをインストール
actual:    rbenvを使ってシステムワイドにrubyをインストールする方法

generated: javaのフレームワークとライブラリってどう違うのか?
actual:    フレームワークとライブラリの違い

generated: vimの便利機能ベスト25
actual:    脱初心者を目指すなら知っておきたい便利なvimコマンド25選(vimmerレベル診断付き)

generated: jsoupの便利な使い方
actual:    jsoup使い方メモ

generated: gitコンフリクトが発生したら焦って何をすれば良いのか
actual:    コンフリクト発生!焦らず利用する4つのコマンド#git

generated: iosのライセンス体系
actual:    iosライセンス&配布方法まとめ

generated: oauth 2.0のエンドポイントを実装する
actual:    oauth 2.0の勉強のため

## 以降は次のNotebookと同じ内容のため説明は省略

https://github.com/sonoisa/qiita-title-generation/blob/main/T5_ja_qiita_title_generation.ipynb

In [24]:
qiita_body = """
AIの進歩はすごいですね。

今回は深層学習を用いて、記事（Qiita）のタイトルを自動生成してくれるAIさんを試作してみました。
この実験は自然言語処理について新人さんに教えるためのハンズオンネタを探索する一環で行ったものになります。

作ったAIは、Qiitaの記事本文（少し前処理したテキスト）を与えると、適したタイトル文字列を作文して返してくれるというものです。

なお、学習データは（2019年頃に）Qiitaの殿堂を入り口にして、評価の高い記事（いいねが50個以上）をスクレイピングしたものを使いました。
つまりヒットした記事のタイトルの付け方を学んだAIであるといえます。

* もう少し詳細:
  * 学習データの例:
    * 入力: "body: hiveqlではスピードに難を感じていたため、私もprestoを使い始めました。 mysqlやhiveで使っていたクエリ..."
    * 出力: "title: hadoop利用者ならきっと知ってる、hive/prestoクエリ関数の挙動の違い"
  * 学習方法: 独自に作った日本語T5の事前学習モデルをこの学習データを用いて転移学習

以下、結果（抜粋）です。generatedが生成されたもの、actualが人が付けたタイトルです。
"""

In [25]:
preprocess_qiita_body(qiita_body)

'body: aiの進歩はすごいですね。 今回は深層学習を用いて、記事(qiita)のタイトルを自動生成してくれるaiさんを試作してみました。 この実験は自然言語処理について新人さんに教えるためのハンズオンネタを探索する一環で行ったものになります。 作ったaiは、qiitaの記事本文(少し前処理したテキスト)を与えると、適したタイトル文字列を作文して返してくれるというものです。 なお、学習データは(2019年頃に)qiitaの殿堂を入り口にして、評価の高い記事(いいねが50個以上)をスクレイピングしたものを使いました。 つまりヒットした記事のタイトルの付け方を学んだaiであるといえます。 *もう少し詳細: *学習データの例: *入力:\xa0"body:\xa0hiveqlではスピードに難を感じていたため、私もprestoを使い始めました。\xa0mysqlやhiveで使っていたクエリ..." *出力:\xa0"title:\xa0hadoop利用者ならきっと知ってる、hive/prestoクエリ関数の挙動の違い" *学習方法:\xa0独自に作った日本語t5の事前学習モデルをこの学習データを用いて転移学習 以下、結果(抜粋)です。generatedが生成されたもの、actualが人が付けたタイトルです。'

In [26]:
MAX_SOURCE_LENGTH = 512  # 入力される記事本文の最大トークン数
MAX_TARGET_LENGTH = 64   # 生成されるタイトルの最大トークン数

# 推論モード設定
trained_model.eval()

# 前処理とトークナイズを行う
inputs = [preprocess_qiita_body(qiita_body)]
batch = tokenizer.batch_encode_plus(
    inputs, max_length=MAX_SOURCE_LENGTH, truncation=True, 
    padding="longest", return_tensors="pt")

input_ids = batch['input_ids']
input_mask = batch['attention_mask']
if USE_GPU:
  input_ids = input_ids.cuda()
  input_mask = input_mask.cuda()

# 生成処理を行う
outputs = trained_model.generate(
    input_ids=input_ids, attention_mask=input_mask, 
    max_length=MAX_TARGET_LENGTH,
    temperature=1.0,            # 生成にランダム性を入れる温度パラメータ
    num_beams=10,               # ビームサーチの探索幅
    diversity_penalty=1.0,      # 生成結果の多様性を生み出すためのペナルティ
    num_beam_groups=10,         # ビームサーチのグループ数
    num_return_sequences=10,    # 生成する文の数
    repetition_penalty=1.5,     # 同じ文の繰り返し（モード崩壊）へのペナルティ
)

# 生成されたトークン列を文字列に変換する
generated_titles = [tokenizer.decode(ids, skip_special_tokens=True, 
                                     clean_up_tokenization_spaces=False) 
                    for ids in outputs]

# 生成されたタイトルを表示する
for i, title in enumerate(generated_titles):
  print(f"{i+1:2}. {postprocess_title(title)}")

/usr/local/lib/python3.7/dist-packages/transformers/generation_beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  "Passing `max_length` to BeamSearchScorer is deprecated and has no effect. "


 1. qiitaのタイトル自動生成ai
 2. 深層学習で記事のタイトル自動生成ai
 3. 深層学習で記事のタイトル自動生成aiを作ろう
 4. 【自動生成】深層学習で記事のタイトル自動生成ai
 5. aiで記事タイトル自動生成(qiita)
 6. 【深層学習】qiitaのタイトル自動生成ai
 7. aiで記事タイトル自動生成
 8. qiitaのタイトル自動生成ai(深層学習)
 9. qiitaのタイトル自動生成ai
10. 【自動生成】qiitaのタイトル自動生成ai
